<a href="https://colab.research.google.com/github/yesid-acm/Aplicaciones_transformers/blob/main/Modelo_sentimientos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ## instalar librerias
!pip install torch
!pip install tensorboard
!pip install -q kaggle
!pip install folium==0.2.1
# requiere la version folium==0.2.1
!pip install -q transformers datasets




import pandas as pd
import numpy as np
import random
import collections
import shutil
import os 

import torch
import transformers
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from transformers import BertTokenizer, BertModel  , BertForMaskedLM, BertForSequenceClassification 

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score, matthews_corrcoef

import matplotlib.pyplot as plt
import seaborn as sns
import gc
    

%load_ext tensorboard

     |████████████████████████████████| 69 kB 5.7 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=7d193a1a0652888ed564ef795b2bdc3b81130aa9924164f5b2eba6d5e146c9c7
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 3.8 MB 10.8 MB/s 
     |████████████████████████████████| 325 kB 46.1 MB/s 
     |████████████████████████████████| 895 kB 43.9 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 6.5 MB 13.3 MB/s 
     |████████████████████████████████| 67 kB 4.2 MB/s 
     |████████████████████████████████| 212 kB 42.4 MB/s 
     |████████████████████████████████| 1.1 MB 14.0 MB/s 
     |████████████████████████████████| 134 kB 

In [2]:
from datasets import load_dataset
from google.colab import files
from transformers import AutoTokenizer

# se usa 'sequence' porque es con base a una 'secuencia de datos, textos
from transformers import AutoModelForSequenceClassification
from datasets import load_metric  # cargar metricas
from transformers import TrainingArguments
from transformers import Trainer #entrenador

In [3]:
#uso de GPU
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
!nvidia-smi

Tue Mar 15 19:23:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### login en HFace

In [5]:
# autenticación en Hugging Face
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [6]:
# config Git e instalar lfs para cargar files pesados.
!apt install git-lfs
!git config --global user.email "yesimera18@gmail.com"
!git config --global user.name "alexhf90"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 0s (17.7 MB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155335 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


# carga dataset

fuente data: https://www.kaggle.com/ricardomoya/criticas-peliculas-filmaffinity-en-espaniol

In [7]:
# descargar archivo Kaggle.json que está en la cuenta de Kaggle, es un token
# cargarlo aquí, verificador de cuenta.
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yesid12","key":"8b33baa9f5cbac414b67f37b7098c210"}'}

In [9]:
#%%bash
# Descarga desde Kaggle
# 1. quitar comentario de la siguiente linea para crear carpeta
#!mkdir ~/.kaggle

# Copiar file
!cp kaggle.json ~/.kaggle/
# cambia los permisos del file
!chmod 600 ~/.kaggle/kaggle.json
#listar
!kaggle datasets list

# descargar dataset a la carpeta /content
!kaggle datasets download -d ricardomoya/criticas-peliculas-filmaffinity-en-espaniol

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                              title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
prasertk/healthy-lifestyle-cities-report-2021                    Healthy Lifestyle Cities Report 2021                 2KB  2022-03-03 00:26:02           1733         64  1.0              
piterfm/2022-ukraine-russian-war                                 2022 Ukraine Russia War                            1007B  2022-03-15 11:11:17           1348        113  1.0              
equinxx/spotify-top-50-songs-in-2021                             Spotify top 50 songs in 2021                         4KB  2022-03-09 08:46:59            729         33  1.0              


In [10]:
#Aquí descomprimo la data .zip
!mkdir data_peli_criticas

#comando para descomprimir data
!unzip /content/criticas-peliculas-filmaffinity-en-espaniol.zip -d ./data_peli_criticas

Archive:  /content/criticas-peliculas-filmaffinity-en-espaniol.zip
  inflating: ./data_peli_criticas/reviews_filmaffinity.csv  


In [11]:
# Leer data 
data_cruda = load_dataset('csv', data_files=['/content/data_peli_criticas/reviews_filmaffinity.csv'], sep='\|\|', split='train')

Using custom data configuration default-c65d66a523ab3d28


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c65d66a523ab3d28/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


In [12]:
data_cruda

Dataset({
    features: ['film_name', 'gender', 'film_avg_rate', 'review_rate', 'review_title', 'review_text'],
    num_rows: 8603
})

## Partición train-test

In [13]:
data_train_test = data_cruda.train_test_split(test_size=0.20)
data_train_test

DatasetDict({
    train: Dataset({
        features: ['film_name', 'gender', 'film_avg_rate', 'review_rate', 'review_title', 'review_text'],
        num_rows: 6882
    })
    test: Dataset({
        features: ['film_name', 'gender', 'film_avg_rate', 'review_rate', 'review_title', 'review_text'],
        num_rows: 1721
    })
})

In [14]:
# ejemplo
data_train_test["train"][2]

{'film_avg_rate': '5,5',
 'film_name': 'Alatriste',
 'gender': 'Aventuras',
 'review_rate': 3.0,
 'review_text': 'De esta película podría decirse algo parecido a aquello del Cid: qué buena película, si tuviera buen director (o productor, o lo que sea). No sé, alguien que se diera cuenta de las dos obviedades que saltan a la vista absolutamente para todo el mundo:Primero, que Viggo Mortensen no habla como un español. En unas escenas parece un argentino ronco, en otras un gringo algo pasado de sangría. Pero bajo ningún concepto pasa, ni remotamente, por español. Por mucha planta que tenga, con semejante acento no puede ni aproximarse al personaje de Alatriste.Segundo, que no se pueden condensar tantas novelas en una sola película.Estos dos detalles, que son evidentes para cualquiera, por alguna razón no le parecieron dignos de consideración a los responsables del proyecto. Uno de los misterios, otro más, del cine español.Alatriste: la triste historia de cómo unos incapaces desaprovecharo

In [15]:
from datasets import ClassLabel
import random
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    
    assert num_examples <= len(dataset), "El número de filas a mostrar no puede ser mayor que el número de filas en el dataset."
    picks = []
    
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [16]:
show_random_elements( data_train_test['test'])


,film_name,gender,film_avg_rate,review_rate,review_title,review_text
0,Torrente 3,Comedia,"3,7",8.0,Para pasar un buen rato.,"Quiero empezar diciendo que no entiendo cómo hay personas que si han visto Torrente 1 y 2 y no les ha gustado, cómo pierden el tiempo en ver la tercera sabiendo que les espera más de lo mismo. Defectos que considero tiene Torrente 3, lo primero demasiados frikis para mi gusto, el argumento no está muy currado que digamos, creo también que tanto el personaje de su hijo (protagonizado por Carlos Latre) como el de su madre (Tony Leblanc) sobran en esta tercera entrega. Por lo demás, tanto Torrente 3 como sus antecesoras son idóneas si te quieres partir de la risa junto con tus amigos, pues el hecho de que Torrente sea un personaje anti-perfecto (egoísta, fanfarrón, racista, corrupto, etc...) es precisamente por lo que hace reír tanto, pues si fuera un policía responsable, humilde y honrado sería una película de ciencia ficción y no una comedia."
1,El otro lado de la cama,Musical,"6,4",8.0,El lado mas musical del cine español,"¿Podeis decirme una película musical española (actual)? Seguro el 99'99% saltaria con: En otro aldo de la cama, por algo será. Apuesta arriesgadisima, que pasa con nota el examen de Comedia Española. Tiene un buen guión, y es muy divertida en todos sus aspectos.Con un reparto buenisimo y unos secundario estelares: Alberto San Juan, y Maria Esteve dan aun mas vida a esta comedia fresca y de enrredos.De la parte musical, la mejor: Te echo de menos (Kiko Veneno) y la interpretación de Alberto San Juan en ""Las chicas son guerreras""."
2,Ocho apellidos vascos,Comedia,"6,0",6.0,Euskadi tiene un color especial,"Me resulta difícil escribir la crítica a esta película, ya que si bien consigue su objetivo que es entretener y hacer reír, la calidad de la misma brilla por su ausencia.Nos encontramos ante una trama que de fácil y previsible asusta, no sorprende en ningún momento una historia básica en una comedia de este tipo. Lo que tiene que la hace especial es que consigue que nos riamos con ella, y, en algunas escenas, mucho. No me parece, como mucha gente piensa, que te estés riendo de principio a fin, más bien tiene algunas escenas graciosas y otras que al menos no se hacen pesadas.En cuanto a las interpretaciones tampoco es destacable nada, en general poca calidad en la interpretación, con algunos momentos puntuales que tiene cada actor. Quien sí me parece acertada toda la cinta es Carmen Machi. En el apartado positivo hay que destacar muy por encima de todo que haciendo una presentación de tópicos sobre vascos y andaluces ninguno de los dos se moleste, más bien se agradece que, por una vez, seamos capaces de reírnos de nosotros mismos, algo que falta nos hace en este tiempo. Esa es la mayor virtud de esta película, jugar con los tópicos hasta dejarlos totalmente exprimidos.Por eso me es difícil hacer esta crítica. Es una película más bien mala pero que te hace reír y entretenerte prácticamente todo el tiempo. Si ese era su objetivo (que todo parece indicar que sí), enhorabuena. De todas formas recomiendo a todo el que quiera (y aún no haya ido al cine a verla) que la vea y que la disfrute, pero que vaya al cine sin pensar que es la película definitiva, más bien, vayan al cine a, simplemente, disfrutar y echarse unas risas."
3,Lo imposible,Drama,"6,8",5.0,Impossible is nothing,"Lo imposible, la película más increíble del 2012 a pesar de estar basada en hechos reales.En cierto modo, me recordó a Más allá de la vida, de Clint Eastwood: a partir del tsunami, fui incapaz de tragármela entera, pese a su calidad visual. De repente, el mundo entero desapareció, y sólo existía una familia separada sobreviviendo en las circunstancias más adversas y cruelmente soporíferas.Realmente, lo poco o mucho que emocione, no lo consigue por mérito propio. A parte de los recursos ya mencionados (música impactante, primeros planos de sus expresiones faciales, heridas desgarradoras, gritos histéricos, abrazos de 

In [17]:
# Convertir a pandas
data_train_test.set_format("pandas")
df_train = data_train_test['train'][:]
df_train.head(3)

,film_name,gender,film_avg_rate,review_rate,review_title,review_text
0,Días de fútbol,Comedia,"6,1",7.0,Comedia española = buena receta,"¿Qué esperaba?Una típica comedia española, reí..."
1,Ocho apellidos catalanes,Comedia,"4,4",5.0,Bigarren parteak ez ziren onak inoiz izan,Secuela bastante más inferior que su predeceso...
2,Alatriste,Aventuras,"5,5",3.0,De cómo Yanes y compañía se cargaron una ocasi...,De esta película podría decirse algo parecido ...


### clasificación de comentarios Positivo/Negativo

* Positivo: $>7$ puntos
* Negativo : $<4$ puntos

In [18]:
df_train['gender'].value_counts()

Comedia       1972
Drama         1349
Terror         745
Aventuras      729
Thriller       633
Fantástico     408
Cine negro     333
Animación      237
Romance        223
Intriga        197
Musical         56
Name: gender, dtype: int64

In [19]:
df_train['review_rate'].value_counts()

8.0     1041
7.0     1029
6.0     1006
5.0      764
9.0      659
3.0      562
1.0      514
4.0      478
2.0      430
10.0     397
Name: review_rate, dtype: int64

In [20]:
# volver al formato 
data_train_test.reset_format()

In [21]:
for omitir in [4,5,6,7, np.nan, None]:
  data_train_test = data_train_test.filter(lambda x : x["review_rate"] != omitir)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

...

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
data_train_test

DatasetDict({
    train: Dataset({
        features: ['film_name', 'gender', 'film_avg_rate', 'review_rate', 'review_title', 'review_text'],
        num_rows: 3603
    })
    test: Dataset({
        features: ['film_name', 'gender', 'film_avg_rate', 'review_rate', 'review_title', 'review_text'],
        num_rows: 892
    })
})

In [23]:
# rating a segmentar en 1 : positivo y 0: negativo 
set(data_train_test['train']['review_rate'])

{1.0, 2.0, 3.0, 8.0, 9.0, 10.0}

In [24]:
# agregar columna labels

def merge_review_rating(examples):
    if examples["review_rate"] <= 3:
        label = 0

    else:
        label = 1
    return {"labels": label}

In [25]:
data_train_test = data_train_test.map(merge_review_rating)

  0%|          | 0/3603 [00:00<?, ?ex/s]

  0%|          | 0/892 [00:00<?, ?ex/s]

In [26]:
show_random_elements(data_train_test['test'] )

,film_name,gender,film_avg_rate,review_rate,review_title,review_text,labels
0,El bola,Drama,"7,0",8.0,ESTO SI TE ATRAVIESA EL ALMA.,"Haré un corto comentario sobre esta película, como padre y como apasionado del cine, quizá mas como lo primero que como lo segundo.Decir que es muy real es una paradoja, efectivamente, es demasiado real, y la mayor virtud que tiene es contarnos las cosas tal cual, como son, y como estan pasando... desgraciadamente este tema y otros de similares caracteristicas se repiten demasiado a menudo y está muy bien una película de este corte de vez en cuando para remover un poco las conciencias y decir BASTA YA!!!.Juan José ballesta es un animal cinematográfico, ha nacido para tal fin, y si no se estropea su carrera (ha pasado a otros y puede pasarle a él), puede llegar muy alto, mas que notable su personaje, por naturalidad y registros... el resto ¿joder, de donde coño se los han sacado ?, porque son absolutamente auténticos, sobre todo el padre del Bola, da miedo desde el primer fotográma, y todo el elenco, muy acertado, de verdad...Escalofriante la escena cúlmine del film, donde se muestra todo el odio y sin sentidos del ser humano, cuesta volver a verla y que no se te mojen los ojos, demasiado dura, demasiado fuerte, demasiado real... y cuesta alabar una secuencia de estas caracteristicas pues entran en conflicto tus valores cinematográficos y los tuyos personales, pero desde luego indiferente no te deja.Técnicamente, la película, está correcta, bien narrada y con sensibilidad... no mucho más que decir, los meritos son otros, los de atravesarte el corazón y la conciencia, es lamentable y condenable todas esta situaciones, y como padre que ama y adora a su hija y a su familia, desde aquí mi más ferviente rechazo y condena a todos esos seres, que se parecen poco a los seres humanos, y me gustaría que existiese una ley mas dura para todos estos CRIMINALES.SALUDOS.",1
1,Alatriste,Aventuras,"5,5",3.0,Muy floja,"...Muy mal dirigida...y ese acento de Viggo Mortensen....uffff...difícil de digerir. Lenta y aburrida, predecible...La película más cara del cine español???? y dónde está el dinero???",0
2,Regresión,Thriller,"5,3",1.0,¡DEVOLUCIÓN!,"Así es como debería subtitularse este bodrio, y es que cuando salí de la sala de cine lo único que me pedía el cuerpo es que me devolvieran la entrada. Y es que me sentí totalmente estafado, menos mal que fue un miércoles a última hora y no había mucha gente, no quisiera imaginarme tener que haber hecho cola o que me hubiese tocado una localidad incomoda para ver este bodrio. Pero Dios mío!, ¿en que estaba pensando Amenábar?, nada tiene ni pies ni cabeza, ni el guión, ni la ambientación, ni los actores, Ethan Hawke, sobreactúa mas que José Luis López Vázquez, (mas quisiera parecerse a él), y que me dicen de Emma Watson, simplemente da pena, es ridícula, hay una escena en la que aparece justo después de que le hayan puesto sobre la cara lagrimas artificiales que ruedan por su cara, y ni por esas, no es capaz de expresar ni un solo sentimiento, de pequeña haciendo Harry Potter tenía su gracia, pero ahora… Me llevé toda la película esperando el final sorpresa e impactante que pensé que tenía que llegar, pero cuando veo que aparecen los títulos de crédito finales, se me cayeron los palos del sombrajo, incluso me quedé en la butaca un rato mas, esperando que saliera la escena final que estaba ahí escondida entre las letras que salían, pero no, ¡que desastre!. En fin, dejo ya de escribir, porque no merece la pena ni el tiempo que estoy gastando con esto. En definitiva si quieren ver a un Amenábar misterioso, vuelvan a alquilar Tesis o Los otros, porque esto..vaya tela",0
3,Los crímenes de Oxford,Intriga,"5,7",2.0,Petardazo,"Mira que hace tiempo que ví la película, y sigo sin comprender que haya gente a la que le haya gustado... me gustaría que me lo explicasen porque yo, aparte de que no entendí nada (y sí, antes me había leído el Codigo Da Vinci y había visto la película) no me

# Modelos

In [27]:
# elegir modelo
modelo_ckpoint = "BSC-TeMU/roberta-base-bne" #"PlanTL-GOB-ES/roberta-base-bne"

## Tokenizar data

In [28]:
# identifica el tokenizardor a partir del modelo que se le dá.
tokenizer = AutoTokenizer.from_pretrained(modelo_ckpoint)#, use_fast=True)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/613 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/497k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [29]:
# tokenizar reseñas
def tokenize_comentarios(examples):
    return tokenizer(examples["review_text"], truncation=True)

In [30]:
# aplicamos la función para tokenizar 
# sólo se deja los labels y el texto
tokenizado_train_test = data_train_test.map(tokenize_comentarios, 
                                            batched=True, 
                                            remove_columns=["film_name", 
                                                            "gender", 
                                                            "film_avg_rate",
                                                            "review_rate",
                                                            "review_title",
                                                            "review_text"
                                                            ])
  

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [31]:
tokenizado_train_test

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 3603
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 892
    })
})

## Cargar modelo preentrenado

In [32]:
# se carga el modelo 
# se usa 'sequence' porque es con base a una 'secuencia de datos, textos
num_labels = 2  # neuronas del output 
modelo_preentrenado = AutoModelForSequenceClassification.from_pretrained(modelo_ckpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at BSC-TeMU/roberta-base-bne were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-bne and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bi

In [33]:
#METRICAS
metricas = load_metric("accuracy", "f1")
metricas

Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions: Predicted labels, as returned by a model.
    references: Ground truth labels.
    normalize: If False, return the number of correctly classified samples.
        Otherwise, return the fraction of correctly classified samples.
    sample_weight: Sample weights.
Returns:
    accuracy: Accuracy score.
Examples:

    >>> accuracy_metric = datasets.load_metric("accuracy")
    >>> results = accuracy_metric.compute(references=[0, 1], predictions=[0, 1])
    >>> print(results)
    {'accuracy': 1.0}
""", stored examples: 0)

In [34]:
def calculo_metricas(eval_pred):

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
  
    return metricas.compute(predictions=predictions, references=labels)

## Fine-tunning del modelo seleccionado

In [35]:
# crear carpeta de resultados del entrenamiento
!mkdir Clasificacion_sentimientos

In [36]:
# ajustes paramétricos
name_modelo = "Clasificacion_sentimientos" #modelo_ckpoint.split("/")[-1]

# hiper-parametricos
batch_size = 4
num_train_epochs=4

# colocar el número de ejemplos existentes en train
num_train_samples = tokenizado_train_test['train'].shape[0]

# dataset train para pasar al Trainer
train_dataset = tokenizado_train_test["train"].shuffle(seed=42).select(range(num_train_samples))

# cada cuanto dar los loggs 
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

# ajuste de los hiperparametro
training_args = TrainingArguments(
                                  output_dir = "Clasificacion_sentimientos",
                                  num_train_epochs = num_train_epochs,     
                                  learning_rate = 2e-5,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  load_best_model_at_end = True,
                                  metric_for_best_model = "accuracy",
                                  weight_decay = 0.01,
                                  evaluation_strategy = "epoch",
                                  save_strategy = "epoch", 
                                  logging_steps = logging_steps,
                                  push_to_hub = True
                                  #push_to_hub_model_id = f"{name_modelo}-finetuned- Críticas películas filmaffinity"
                              )

In [37]:
## instanciación de entrenamiento
trainer = Trainer(
                  model = modelo_preentrenado, # modelo sequencia cargado anteriormente
                  args = training_args, 
                  compute_metrics = calculo_metricas,
                  train_dataset = train_dataset,
                  eval_dataset =  tokenizado_train_test["test"],
                  tokenizer = tokenizer   )

Cloning https://huggingface.co/alexhf90/Clasificacion_sentimientos into local empty directory.


In [38]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3603
  Num Epochs = 4
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3604


Epoch,Training Loss,Validation Loss,Accuracy
1,0.256600,0.529937,0.893498
2,0.096300,0.288527,0.938341
3,0.013300,0.354578,0.940583
4,0.000200,0.339877,0.942825


***** Running Evaluation *****
  Num examples = 892
  Batch size = 4
Saving model checkpoint to Clasificacion_sentimientos/checkpoint-901
Configuration saved in Clasificacion_sentimientos/checkpoint-901/config.json
Model weights saved in Clasificacion_sentimientos/checkpoint-901/pytorch_model.bin
tokenizer config file saved in Clasificacion_sentimientos/checkpoint-901/tokenizer_config.json
Special tokens file saved in Clasificacion_sentimientos/checkpoint-901/special_tokens_map.json
tokenizer config file saved in Clasificacion_sentimientos/tokenizer_config.json
Special tokens file saved in Clasificacion_sentimientos/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 892
  Batch size = 4
Saving model checkpoint to Clasificacion_sentimientos/checkpoint-1802
Configuration saved in Clasificacion_sentimientos/checkpoint-1802/config.json
Model weights saved in Clasificacion_sentimientos/checkpoint-1802/pytorch_model.bin
tokenizer config file saved in Clasificacion_sentim

TrainOutput(global_step=3604, training_loss=0.11437740890783663, metrics={'train_runtime': 2565.6151, 'train_samples_per_second': 5.617, 'train_steps_per_second': 1.405, 'total_flos': 2970959078811060.0, 'train_loss': 0.11437740890783663, 'epoch': 4.0})

# Mandas a Hugging Face HUB

In [39]:
# push 
trainer.push_to_hub()

Saving model checkpoint to Clasificacion_sentimientos
Configuration saved in Clasificacion_sentimientos/config.json
Model weights saved in Clasificacion_sentimientos/pytorch_model.bin
tokenizer config file saved in Clasificacion_sentimientos/tokenizer_config.json
Special tokens file saved in Clasificacion_sentimientos/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/476M [00:00<?, ?B/s]

Upload file runs/Mar15_19-31-38_6229a656de94/events.out.tfevents.1647372798.6229a656de94.70.0:  34%|###4      …

remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified
remote: tput: No value for $TERM and no -T specified        
To https://huggingface.co/alexhf90/Clasificacion_sentimientos
   70dd028..ee3065c  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9428251121076233}]}
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified
remote: tput: No value for $TERM and no -T specified
To https://huggingface.co/alexhf90/Clasificacion_sentimientos
   ee3065c..750f02d  main -> main



'https://huggingface.co/alexhf90/Clasificacion_sentimientos/commit/ee3065c01fc706eb89d6ecbed009f2ff52c4a275'